In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import norm
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
import statsmodels as sm
%matplotlib inline

# Lesson 11: t-Tests, Part3

## Independent Samples t-test thoughts

Pooled Variance vs Total Standard Error

**Pooled Variance**

$$SS_x = \sum({x_i - \bar{x}})^2$$
$$SS_y = \sum({y_i - \bar{y}})^2$$

$$S_p^2 = \frac{SS_x + SS_y}{df_x + df_y}$$

Assumptions
1. $X$ and $Y$ are from random samples from two independent populations
1. Populations that X and Y come from should be approximtely normal
1. Sample data can estimate population variances
1. Population variances should be roughly equal



**Standard Error (total)**
$$S_{\bar{x} - \bar{y}} = \sqrt{\frac{S_p^2}{n_x} + \frac{S_p^2}{n_y}}$$

## Pooled Variance

In [173]:
x = [5, 6, 1, -4]
y = [3, 7, 8]
mean_x = np.mean(x)
mean_y = np.mean(y)
sigma_x = np.std(x, ddof=1)
sigma_y = np.std(y, ddof=1)
mean_x, mean_y

(2.0, 6.0)

#### Calculate Pooled Variance: $S_p^2$

Pooled Variance:
$$S_p^2 = \frac{SS_1 + SS_2}{df_1 + df_2}$$

SS = sum of squared differences (**this isn't $\sigma$ or $\sigma^2$ because you're not dividing by $n$ samples**)

In [196]:
pooled_variance = (62 + 14)/5.0
pooled_variance

15.2

Pooled variance is: 15.2

$$SE = \sqrt{\frac{S_p^2}{n_1}+\frac{S_p^2}{n_2}}$$

#### Calculate Corrected Standard Error

$$se_{x-y} = \sqrt{\frac{S_p^2}{n_1}+\frac{S_p^2}{n_2}}$$

In [199]:
se_corrected = np.sqrt(pooled_variance/len(x) + pooled_variance/len(y))
se_corrected

2.9776948578836393

In [182]:
np.std(x)**2

15.5

#### Calculate t-statistic

Recall: t-statistic
$$t = \frac{(\bar{x} - \mu)}{se}$$

But now let's apply it to the diffrence of sample means and the population means ( of which we expect none).  Our standard error will be the 'corrected standard error' calculated from the $x$'s and $y$'s

$$t_{statistic} = \frac{(\bar{x} - \bar{y}) - (\mu_x - \mu_y)}{S_{\bar{x} - \bar{y}}}$$

In [200]:
(mean_x - mean_y)/float(se_corrected)

-1.343320988518935

#### Calculate t-critical value

Recall: t-critical is the value at which the t_statistic because significant

$$\alpha = .05$$
$$SE = 2.978$$

In [203]:
dof = len(x) + len(y) -2
sp.stats.t.ppf(1- .05/2.0,dof) 

2.5705818366147395

#### Retain Null

## Shoe Quantities: Male vs Female

Who has more shoes, males or females

#### Null Hypothesis

$$H_0\; : \bar{x}_m = \bar{x}_f$$
$$H_A\; : \bar{x}_m \neq \bar{x}_f$$

#### Import data

In [92]:
shoes = pd.DataFrame.from_csv('/Users/ryanlambert/Downloads/Shoes - Lesson 11 - Sheet1.csv')
shoes = shoes.reset_index()

#### Calculate summary statistics

In [134]:
mean_female = shoes['Pairs of shoes (females)'].mean()
mean_male = shoes['Pairs of shoes (males)'].mean()
sigma_female = shoes['Pairs of shoes (females)'].std(ddof=1)
sigma_male = shoes['Pairs of shoes (males)'].std(ddof=1)
n_female = shoes['Pairs of shoes (females)'].count()
n_male = shoes['Pairs of shoes (males)'].count()
dof_female = n_female - 1
dof_male = n_male - 1
{
    "mean_female":mean_female, 
    "mean_male":mean_male, 
    "sigma_female":sigma_female, 
    "sigma_male":sigma_male, 
    "n_female":n_female, 
    "n_male":n_male, 
    "dof_female":dof_female,
    "dof_male":dof_male
}

{'dof_female': 6,
 'dof_male': 10,
 'mean_female': 33.142857142857146,
 'mean_male': 18.0,
 'n_female': 7,
 'n_male': 11,
 'sigma_female': 31.360423952430722,
 'sigma_male': 34.272437905699093}

#### Calculate t-statistic

why are we using t-statistic?  

We're using t-statistic because we're inferring something about a population even though we have no data on the population.  The t-table is more conservative.  (particularly that and the degrees of freedom use instead of $n$)

recall: when doing t-statistic between to populations based on one sample from each population we must include standard error from both populations.  They're combined thusly: 

$$SE_{total} = \sqrt{\frac{\sigma_1^2}{n_1}+\frac{\sigma_2^2}{n_2}}$$

In [135]:
se = np.sqrt((sigma_male**2/n_male) + (sigma_female**2/n_female))
se

15.725088769901236

recall:
$$t = \frac{\bar{x} - \mu}{SE}$$

In [136]:
t_statistic = (mean_female - mean_male)/se
t_statistic

0.96297435037959744

#### Calculate P-Value

Recall 
$$p = f(t, dof_{total})$$

In [137]:
dof_total = dof_male + dof_female

Recall
t-table returns area under the curve left of this point.  Since t is positive and we're interested in the

In [140]:
(1 - sp.stats.t.cdf(t_statistic, dof_total)) * 2

0.34988961442139677

$$ p = .359$$

#### Calculate Critical P-value

recall: 
$$p = f(alpha, dof)$$

$$dof_{total} = dof_{male} + dof_{female}$$

In [139]:
p_critical = sp.stats.t.ppf(.025, dof_total)
p_critical

-2.1199052992210112

#### Calculate confidence interval 

Recall: Confidence interval is an upper and lower bound.  

$$upperbound = \bar{x} + E_{margin}$$
$$lowerbound = \bar{x} - E_{margin}$$
$$margin = f(alpha, se)$$

Confidence intervals are two sided

In [154]:
confidence = .95
alpha = .05
margin = se * sp.stats.t.ppf(1 - alpha/2.0, dof_total)
margin, se, alpha, sp.stats.t.ppf(1 - alpha/2.0, dof_total)

(33.335699014034439, 15.725088769901236, 0.05, 2.1199052992210112)

In [166]:
mean_difference = mean_female - mean_male
mean_difference

15.142857142857146

both standard errors contribute to the standard error of the differences in the means.  

Our confidence interval will be applied to the mean difference.  

$$ \bar{x} \pm se*t(confidence\%, dof_{total})$$

In [167]:
upperbound_mean_difference = mean_difference + se * sp.stats.t.ppf(
    .975, 
    dof_total
    )
lowerbound_mean_difference = mean_difference - se * sp.stats.t.ppf(
    .975, 
    dof_total
    )
lowerbound_mean_difference, upperbound_mean_difference

(-18.192841871177293, 48.478556156891585)

In [168]:
mean_difference, t_statistic, se

(15.142857142857146, 0.96297435037959744, 15.725088769901236)

#### Calculate $ R^2$

recall: $$R^2 = \frac{t^2}{(t^2 + dof)}$$

In [169]:
r_squared = t_statistic**2/(t_statistic**2 + dof_total)
r_squared

0.05478242400037163

## Acne

Does this drug have an effect on acne or not?  

#### Establish Null Hypothesis

$$H_0 \;: \; \mu_A = \mu_B$$
$$H_A \;: \; \mu_A \neq \mu_B$$

#### Calcuate t-statistic

recall: t-statistic is the number of standard errors one sample mean is from another mean.  

In [86]:
samples_drugA = [40, 36, 20, 32, 45, 28]
samples_drugB = [41, 39, 18, 23, 35]

n_drugA = len(samples_drugA)
n_drugB = len(samples_drugB)

dof_drugA = n_drugA - 1
dof_drugB = n_drugB - 1
dof_AandB = dof_drugA + dof_drugB

se = np.sqrt(np.std(samples_drugA, ddof = 1)**2/n_drugA + np.std(samples_drugB, ddof=1)**2/n_drugB)
mean_drugA = np.mean(samples_drugA)
mean_drugB = np.mean(samples_drugB)

recall 
$$t = \frac{\mu_A - \mu_B}{SE}$$

also recall for total standard error 

$$SE_{total} = \sqrt{\frac{\sigma_1^2}{n_1}+\frac{\sigma_2^2}{n_2}}$$

You forgot to square the $\sigma$'s

In [87]:
t_statistic = (mean_drugA - mean_drugB)/float(se)
t_statistic

0.39547554497329268

#### Calculate p-value

recall: p-value is the probability of getting this result by chance for a given $\alpha$ or probability

Also recall: 
For two tailed tests we need to get the area under the curve for each side.  So once you get the area under the curve on the left side multiply that by 2.  

In [75]:
(1 - sp.stats.t.cdf(t_statistic, dof_AandB)) * 2


0.67128571750622212

$p = .671$



Results are not significant

#### Calculate *t-critical* values

recall: t-critical values are the limits of significance based on given probabilities of significance.  

i.e. What are the t_statistics above which we would accept the null hypothesis or a given significance or $\alpha$

This is a two tailed test so $\alpha = .05$ equates to .025 on each side of the distribution.  

In [88]:
alpha = .05
sp.stats.t.ppf(alpha/2.0, dof_AandB)

-2.262157162740992

## Food Prices

Find the cheapest town to eat in based on samples from both towns.  

This is a t-test since we don't have population parameters.  

Load raw data into dataFrame

In [6]:
food_prices = pd.DataFrame.from_csv('/Users/ryanlambert/Downloads/Food Prices - Lesson 11 - Sheet1.csv')
food_prices = food_prices.reset_index()

In [19]:
mean_gettysburg, mean_wilma = food_prices.mean()

In [18]:
sigma_gettysburg, sigma_wilma = food_prices.std()

In [26]:
n_gettysburg = food_prices['Average meal prices at restaurants in Gettysburg ($)'].count()

In [25]:
n_wilma = food_prices['Average meal prices at restaurants in Wilma ($)'].count()

#### Null Hypothesis

$$H_0 \; : \; \mu_G = \mu_W$$ 

no difference

$$H_A \; : \; \mu_G \neq \mu_W$$ 

difference

Since we don't know which way the difference will go we'll make this a **two tailed** t-test

#### Calculate Standard Error

Recall:  Standard error is $\frac{\sigma}{\sqrt{n}}$

But since we're comparing two samples that each have their own standard error, the total standard error will be.  

$$SE_{\bar{x}_G - \bar{x}_W} = \sqrt{\frac{\sigma_G^{2}}{n_G} + \frac{\sigma_W^{2}}{n_W}}$$

To_understand: When is this used?  Is it used when we are looking at net differences?  

In [28]:
se = np.sqrt((sigma_gettysburg**2)/n_gettysburg + (sigma_wilma**2)/n_wilma)
se

0.85311008476772265

#### Calculate t-Statistic

recall: t-Statistic is measured in standard errors.  How far in standard errors, a given intervention is from a mean.  

$$\frac{\bar{x} - \mu}{SE}$$

In this case it could be either 

$$\frac{\bar{x_G} - \bar{x_W}}{SE_{\bar{x}_G-\bar{x}_W}}$$

or 

$$\frac{\bar{x_W} - \bar{x_G}}{SE_{\bar{x}_G-\bar{x}_W}}$$

Since we're measuring difference.  

In [30]:
t_statistic = (mean_gettysburg - mean_wilma)/float(se)
t_statistic

-2.5769390582356815

#### Calculate P-Value

recall: P-Value is the probability that this value could have been obtained by chance.  

P-Value is calculated from the t-statistic and $dof$ in this case instead of $n$ because of dependence within the samples and that we're inferring about the population from these samples.  

But what do I use for $dof$ when I clearly have two different $n$'s

$$n_W = 14,\; n_G = 18$$

##### Answer
The $dof$ or $n$ is just the addition of the two of them together 

$$n = n_G + n_W$$

and likewise, because $dof = n - 1$

$$dof_{total} = (dof_G) + (dof_W)$$

$$dof_{total} = (n_G - 1) + (n_W - 1)$$

$$dof_{total} = n_G + n_W -2$$

In [35]:
n_total = n_gettysburg + n_wilma
dof = n_total - 2

In [37]:
sp.stats.t.cdf(t_statistic, dof)

0.0075647325763756549

Because this is a **Two Tailed** test, we're going to get the area on both sides of the curve.  So we multiply this result by two.  

In [40]:
p = sp.stats.t.cdf(t_statistic, dof)*2
p

0.01512946515275131

#### Calculate Critical Value

Recall: Critical value is the t_statistic at which the results are significant.  The critical value is a function of **desired significance** and **standard error**.  The output is **number of standard errors from the mean**

Also recall: since this is a two tailed test, .05 alpha would correspond to .025 on each side of the distribution.  

In [51]:
alpha = .05
t_critical = sp.stats.t.ppf(1 - alpha/2.0, dof)
t_critical

2.0422724563012373

#### Evaluate our Null Hypothesis

In [54]:
print "t-statistic is: ", t_statistic
print "t-critical value is: ", t_critical

t-statistic is:  -2.57693905824
t-critical value is:  2.0422724563


**We reject the Null Hypothesis**

The prices are significantly different at $p = .01513$